#### <img src="../figs/logocimat.png" height="20%" width="20%"  align="center"/>

# <center> Ciencia de Datos<center>

<center> Víctor Muñiz Sánchez <center>
<center> Maestría en Cómputo Estadístico <center>

# Análisis de Componentes Principales (PCA)

## PCA como un método de reducción de dimensión

### Ejemplo 1: Veamos una versión simplificada de los dígitos MNIST

In [ ]:
import numpy as np
import matplotlib.pylab as plt 
from sklearn.datasets import load_digits 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import os

#os.chdir('/home/victor/cursos/ciencia_de_datos_general/')
%matplotlib inline


In [ ]:
digits = load_digits()
print('dimensiones:',digits.data.shape)
j = 1
#np.random.seed(1)
fig = plt.figure(figsize=(1,1)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
for i in np.random.choice(digits.data.shape[0], 25):
    plt.subplot(5,5,j), plt.imshow(np.reshape(digits.data[i,:], (8,8)), cmap='binary'), plt.axis('off')
    j += 1
plt.show()

#### PCA

In [ ]:
ncomp=2
pca_digits=PCA(ncomp)
digits.data_proj = pca_digits.fit_transform(digits.data) 
plt.figure(figsize=(10,10))
plt.scatter(digits.data_proj[:, 0], digits.data_proj[:, 1], lw=0.25, c=digits.target, edgecolor='k',  s=100, cmap=plt.colormaps.get_cmap('cubehelix'))
plt.xlabel('PC1', size=20), plt.ylabel('PC2', size=20), plt.title('2D Projection of handwritten digits with PCA', size=25)
plt.colorbar(ticks=range(10), label='digit value')
plt.clim(-0.5, 9.5)
print('Varianza explicada con %d PC: %.3f'%(ncomp,np.sum(pca_digits.explained_variance_ratio_)))

### Ejemplo 2: Eigenfaces

In [ ]:
from sklearn.datasets import fetch_olivetti_faces 
faces = fetch_olivetti_faces().data
print('dimensiones:',faces.shape)

fig = plt.figure(figsize=(10,10)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
# plot 25 random faces
j = 1
np.random.seed(0)
for i in np.random.choice(range(faces.shape[0]), 25): 
    ax = fig.add_subplot(5, 5, j, xticks=[], yticks=[]) 
    ax.imshow(np.reshape(faces[i,:],(64,64)), cmap=plt.cm.gray, interpolation='nearest') 
    j += 1
plt.show()

#### Usaremos un conjunto de entrenamiento para obtener los componentes principales

In [ ]:
# separar conjunto de entrenamiento y prueba
# tomo la ultima imagen de cada sujeto como dato de prueba
te_ind = np.array([i for i in range(1,410) if (i % 10 == 0)]) -1
tr_ind = np.array([i for i in range(1,400) if (i % 10 != 0)]) -1

training_faces, test_faces = faces[tr_ind,:], faces[te_ind,:]
training_faces.shape

In [ ]:
# guardo las imagenes como png desde arreglos numpy
for i in range(training_faces.shape[0]):
    plt.imsave('../data/faces/faces_64/train/img'+str(tr_ind[i]+1)+'.png',
               np.reshape(training_faces[i,:],(64,64)),cmap='gray')

for i in range(test_faces.shape[0]):
    plt.imsave('../data/faces/faces_64/test/img'+str(te_ind[i]+1)+'.png',
               np.reshape(test_faces[i,:],(64,64)),cmap='gray')

#### Rostros centrados y su varianza
¿Qué pasa si no se estandarizan?

In [ ]:
scaler = StandardScaler(with_mean=True,with_std=True)
faces_scale = scaler.fit_transform(training_faces)
mean_face = np.reshape(scaler.mean_, (64,64))
sd_face = np.reshape(np.sqrt(scaler.var_), (64,64))

plt.figure(figsize=(10,5))
plt.subplot(121), plt.imshow(mean_face, cmap=plt.cm.gray), plt.axis('off'), plt.title('Mean face')
plt.subplot(122), plt.imshow(sd_face, cmap=plt.cm.gray), plt.axis('off'), plt.title('SD face')
plt.show()
faces_scale.shape

#### Hacemos PCA y vemos sus componentes principales. 
¿Cuántos componentes usar?

In [ ]:
n_comp = 50 # numero de componentes
faces_pca = PCA(n_comp)
faces_proj = faces_pca.fit_transform(faces_scale) 

In [ ]:
## varianza explicada
plt.figure(figsize=(10, 8))
plt.plot(np.cumsum(faces_pca.explained_variance_ratio_), linewidth=2)
plt.grid(), plt.axis('tight'), plt.xlabel('n_components'), plt.ylabel('Varianza acumulada')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,5)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1.3, hspace=0.05, wspace=0.05) 
# Graficamos los primeros 10 componentes
for i in range(10): 
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[], title = 'PC '+str(i+1))
    ax.imshow(np.reshape(faces_pca.components_[i,:], (64,64)), cmap=plt.cm.gray, interpolation='nearest')

#### Visualización de los primeros dos eigenfaces. 
Usamos una visualización interactiva con Bokeh. Asegúrate de tener instalado los componentes del módulo

In [ ]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
#from bokeh.palettes import brewer, Viridis256
import re

# para ordenar los archivos según su numeración...
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

dir_tr = '../data/faces/faces_64/train/'
sorted_files = sorted_alphanumeric(os.listdir(dir_tr))
name_imgs_tr = [os.path.join(dir_tr,f) for f in sorted_files]

In [ ]:
import colorcet as cc
import matplotlib.colors as colors
from colorcet.plotting import swatch, swatches, candy_buttons

# cc.glasbey_bw es un mapa de colores para datos categóricos... 
# ver https://colorcet.holoviz.org/user_guide/Categorical.html
lab = np.concatenate([np.repeat(i,9) for i in range(1,41)]) #hay 9 fotos de cada sujeto en entrenamiento
color_map = [colors.rgb2hex(cc.glasbey_bw[i]) for i in lab-1]

# diccionario con los datos para la grafica de Bokeh
pc_source = dict(x=faces_proj[:,1], y=faces_proj[:,2], 
                 label=lab,
                 color=color_map,
                 desc=['sujeto: '+str(tr_ind[i]+1) for i in range(faces_proj.shape[0])],
                 imgs=name_imgs_tr)

In [ ]:
output_file("eigenfaces.html")

source = ColumnDataSource(data = pc_source)

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="100" alt="@imgs" width="100"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@desc</span>
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
    )

p = figure(width=1400, height=800, tools=[hover], title="Eigenfaces",
          x_axis_label="PC 1",y_axis_label="PC 2")
p.circle('x', 'y', size=5,  color='color', source=source)
show(p)

#### Proyectamos los datos de prueba, y vemos su relación con los de entrenamiento

In [ ]:
# proyectar datos de prueba
faces_scale_te = scaler.transform(test_faces)
faces_proj_te = faces_pca.transform(faces_scale_te) 

In [ ]:
dir_te = '../data/faces/faces_64/test/'
sorted_files = sorted_alphanumeric(os.listdir(dir_te))
name_imgs_te = [os.path.join(dir_te,f) for f in sorted_files]

lab_te = np.array(range(1,41))
color_map_te = [colors.rgb2hex(cc.glasbey_bw[i]) for i in lab_te-1]

pc_source_te = dict(x=faces_proj_te[:,1], y=faces_proj_te[:,2],
                    label=lab_te, color=color_map_te,                    
                    desc=['sujeto test'+str(te_ind[i]+1) for i in range(1,faces_proj_te.shape[0])],
                    imgs=name_imgs_te)

In [ ]:
output_file("eigenfaces.html")

source = ColumnDataSource(data = pc_source)
source_te = ColumnDataSource(data = pc_source_te)

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="100" alt="@imgs" width="100"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@desc</span>
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
    )

p = figure(width=1400, height=800, tools=[hover], title="Eigenfaces",
          x_axis_label="PC 1",y_axis_label="PC 2")
p.circle('x', 'y', size=5,  color='color', source=source)
p.circle('x', 'y', size=15,  color='red', source=source_te)
show(p)